# Importation

In [1]:
import pandas as pd
import re
import numpy as np
if not hasattr(np, 'unicode_'):
    np.unicode_ = str
import ast
import os
import math
import plotly.express as px
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Dict, Tuple, List
from scipy.signal import chirp, find_peaks, peak_widths

In [2]:
cosmicai_path = 'Data'

df = pd.read_parquet(f'{cosmicai_path}/bandpass_qa0_no_partitions_labelled_filt.parquet')
df.reset_index(drop=True, inplace=True)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56024 entries, 0 to 56023
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype              
---  ------               --------------  -----              
 0   eb_uid               56024 non-null  object             
 1   cal_data_id          56024 non-null  object             
 2   cal_reduction_id     56024 non-null  object             
 3   start_valid_time     56024 non-null  datetime64[ns, UTC]
 4   receiver_band        56024 non-null  object             
 5   ref_antenna_name     56024 non-null  object             
 6   antenna              56024 non-null  object             
 7   polarization         56024 non-null  object             
 8   sideband             56024 non-null  object             
 9   baseband_name        56024 non-null  object             
 10  spw_name             56024 non-null  object             
 11  spw_name_ms          56024 non-null  object             
 12  frequency_array   

| N    |  Column Name             | Description   |              
| :--- | :----                    |          ---: |
| 0    | eb_uid              | Unique ID for one execution |
| 1    | cal_data_id         | Calibration Data ID  |             
| 2    | cal_reduction_id    | Calibration Reduction ID |
| 3    | start_valid_time    | Start Time for a bandpass observation |
| 4    | receiver_band       | ALMA Band used |
| 5    | ref_antenna_name    | Reference Antenna used for Bandpass calculation |       
| 6    | antenna             | Antenna for which the Bandpass solution is produced |
| 7    | polarization        | Reciever Polarization |
| 8    | sideband            | Reciever Sideband |
| 9    | baseband_name       | Baseband Name |
| 10   | spw_name            | Spectral Window Name |
| 11   | frequency_array     | Array with frequency dimension, Hz |
| 12   | amplitude_corr_tsys | Array with the amplitude dimension, corrected by Tsys (units?) |
| 13   | phase_norm_wrap     | Array with the phase dimension, normalized and wrapped, in radians |

In [4]:
df.head(10)

,eb_uid,cal_data_id,cal_reduction_id,start_valid_time,receiver_band,ref_antenna_name,antenna,polarization,sideband,baseband_name,spw_name,spw_name_ms,frequency_array,amplitude_corr_tsys,phase_norm_wrap,QA2 Flag(s)
0,uid://A002/Xff45a6/X4435,CalData_7,CalReduction_7,2022-10-06 11:04:36.384000+00:00,ALMA_RB_07,DV21,DA41,X,LSB,BB_1,SpectralWindow_68,spw_25,"[351724225880.5871, 351708600880.5871, 3516929...","[0.014256122636602484, 0.014731913342933525, 0...","[-0.06451911942250001, -0.0617732094225, -0.00...",[]
1,uid://A002/Xff45a6/X4435,CalData_7,CalReduction_7,2022-10-06 11:04:36.384000+00:00,ALMA_RB_07,DV21,DA41,Y,LSB,BB_1,SpectralWindow_68,spw_25,"[351724225880.5871, 351708600880.5871, 3516929...","[0.013380656345701562, 0.014427262421258363, 0...","[-0.06347614638691666, -0.01566072838691667, -...",[]
2,uid://A002/Xff45a6/X4435,CalData_7,CalReduction_7,2022-10-06 11:04:36.384000+00:00,ALMA_RB_07,DV21,DA42,X,LSB,BB_1,SpectralWindow_68,spw_25,"[351724225880.5871, 351708600880.5871, 3516929...","[0.014431147004386208, 0.013925552020024039, 0...","[-0.2176890534425, -0.22305674344249998, -0.19...",[]
3,uid://A002/Xff45a6/X4435,CalData_7,CalReduction_7,2022-10-06 11:04:36.384000+00:00,ALMA_RB_07,DV21,DA42,Y,LSB,BB_1,SpectralWindow_68,spw_25,"[351724225880.5871, 351708600880.5871, 3516929...","[0.01401435387584803, 0.014249317856180726, 0....","[0.03815305532075, 0.06564197832074999, 0.0391...",[]
4,uid://A002/Xff45a6/X4435,CalData_7,CalReduction_7,2022-10-06 11:04:36.384000+00:00,ALMA_RB_07,DV21,DA43,X,LSB,BB_1,SpectralWindow_68,spw_25,"[351724225880.5871, 351708600880.5871, 3516929...","[0.014949316205388486, 0.015328749924888039, 0...","[-0.13109463273738334, -0.18370725273738334, -...",[]
5,uid://A002/Xff45a6/X4435,CalData_7,CalReduction_7,2022-10-06 11:04:36.384000+00:00,ALMA_RB_07,DV21,DA43,Y,LSB,BB_1,SpectralWindow_68,spw_25,"[351724225880.5871, 351708600880.5871, 3516929...","[0.01490487109608887, 0.01510182558336035, 0.0...","[-0.16930026337416668, -0.13606859337416666, -...",[]
6,uid://A002/Xff45a6/X4435,CalData_7,CalReduction_7,2022-10-06 11:04:36.384000+00:00,ALMA_RB_07,DV21,DA44,X,LSB,BB_1,SpectralWindow_68,spw_25,"[351724225880.5871, 351708600880.5871, 3516929...","[0.015536372678635918, 0.014857486677088527, 0...","[-0.21134714639021668, -0.26473911639021663, -...",[]
7,uid://A002/Xff45a6/X4435,CalData_7,CalReduction_7,2022-10-06 11:04:36.384000+00:00,ALMA_RB_07,DV21,DA44,Y,LSB,BB_1,SpectralWindow_68,spw_25,"[351724225880.5871, 351708600880.5871, 3516929...","[0.014249162305549411, 0.014134395659441731, 0...","[-0.08561645345641668, -0.02401703545641667, -...",[]
8,uid://A002/Xff45a6/X4435,CalData_7,CalReduction_7,2022-10-06 11:04:36.384000+00:00,ALMA_RB_07,DV21,DA45,X,LSB,BB_1,SpectralWindow_68,spw_25,"[351724225880.5871, 351708600880.5871, 3516929...","[0.01573515363706677, 0.016209123561557178, 0....","[-0.10371428527500001, -0.15062227527499997, -...",[]
9,uid://A002/Xff45a6/X4435,CalData_7,CalReduction_7,2022-10-06 11:04:36.384000+00:00,ALMA_RB_07,DV21,DA45,Y,LSB,BB_1,SpectralWindow_68,spw_25,"[351724225880.5871, 351708600880.5871, 3516929...","[0.013769915632359766, 0.014202971749038364, 0...","[-0.009720573608583334, -0.021781495108583335,...",[]


In [5]:
for length, grp in df.groupby(
        df['frequency_array'].str.len()
    ):
    print('length: ', str(length) + ', size:', len(grp))

length:  64, size: 8346
length:  120, size: 9814
length:  128, size: 5600
length:  240, size: 10808
length:  256, size: 80
length:  480, size: 3096
length:  960, size: 15162
length:  1024, size: 552
length:  1920, size: 2386
length:  2048, size: 180


# Demonstration

In [ ]:
bandpass_sel_df = df.query('eb_uid == "uid://A002/Xff45a6/X4435"').copy()

In [ ]:
fig1 = px.scatter(
    x = bandpass_sel_df.iloc[0]['frequency_array'],
    y = bandpass_sel_df.iloc[0]['amplitude_corr_tsys'],
    render_mode='webgl', range_y=[0, 0.02], 
    labels={'x': 'Frequency [Hz]', 'y': 'Amplitude [?]'}
)
fig1

In [ ]:
fig1 = px.scatter(
    x = bandpass_sel_df.iloc[20]['frequency_array'],
    y = bandpass_sel_df.iloc[20]['phase_norm_wrap'],
    render_mode='webgl', range_y=[-np.pi, np.pi], 
    labels={'x': 'Frequency [Hz]', 'y': 'Phase [rad]'}
)
fig1

In [ ]:
bandpass_explode_df = bandpass_sel_df.explode(['frequency_array','amplitude_corr_tsys','phase_norm_wrap'])

bandpass_explode_df['label'] = bandpass_explode_df.apply(
    lambda x: f'{x["antenna"]}, {x["spw_name"]}, pol {x["polarization"]}', axis=1)

In [ ]:
fig2 = px.scatter(
    bandpass_explode_df,
    x='frequency_array', 
    y='amplitude_corr_tsys',
    color='label', render_mode='webgl',
)
fig2.update_traces(mode='lines', opacity=0.5)

In [ ]:
fig2 = px.line(
    bandpass_explode_df.query('antenna == ["DV24", "DA46", "DV09"] and spw_name == "SpectralWindow_68"'),
    x='frequency_array', 
    y='amplitude_corr_tsys',
    color='label', render_mode='webgl'
)
fig2

In [ ]:
fig2 = px.line(
    bandpass_explode_df.query('antenna == "DA64"'),
    x='frequency_array', 
    y='amplitude_corr_tsys',
    color='label', render_mode='webgl'
)
fig2

In [ ]:
fig2 = px.line(
    bandpass_explode_df.query('antenna == "DV09"'),
    x='frequency_array', 
    y='amplitude_corr_tsys',
    color='label', render_mode='webgl'
)
fig2

In [ ]:
fig2 = px.line(
    bandpass_explode_df.query('antenna == "DA59"'),
    x='frequency_array', 
    y='amplitude_corr_tsys',
    color='label', render_mode='webgl'
)
fig2

In [ ]:
fig2 = px.line(
    bandpass_explode_df.query('antenna == ["DA61", "DV03"]'),
    x='frequency_array', 
    y='phase_norm_wrap',
    color='label', render_mode='webgl'
)
fig2

# Data Analysis

In [6]:
actual_specs = [np.array(x, dtype=float)
                 for x in df['amplitude_corr_tsys'].tolist()]

In [7]:
freqs = [np.array(x, dtype=float)
                 for x in df['frequency_array'].tolist()]

In [8]:
eb_uid = df['eb_uid']
antenna = df['antenna']
spw_name_ms = df['spw_name_ms']
polarization = df['polarization']

In [9]:
length_groups: Dict[int, List[int]] = {}
for i, s in enumerate(actual_specs):
    L = s.shape[0]
    length_groups.setdefault(L, []).append(i)

In [10]:
result: Dict[int, Tuple[np.ndarray, ...]] = {}
for L, idxs in length_groups.items():
    actual_specs_L = np.vstack([actual_specs[i] for i in idxs])
    freqs_L = np.vstack([freqs[i] for i in idxs])
    eb_uid_L = eb_uid[idxs]
    antenna_L = antenna[idxs]
    spw_name_ms_L = spw_name_ms[idxs]
    polarization_L = polarization[idxs]
    result[L] = (actual_specs_L, freqs_L, eb_uid_L, antenna_L, spw_name_ms_L, polarization_L)

In [11]:
print(f"Found lengths: {sorted(result.keys())}")

Found lengths: [64, 120, 128, 240, 256, 480, 960, 1024, 1920, 2048]


In [13]:
lengths, step_means, step_stds, span_means, span_stds = [], [], [], [], []
 
for length in sorted(result):
    actual_specs_L, freqs_L, eb_uid_L, antenna_L, spw_name_ms_L, polarization_L = result[length]
    print('Analyzing channel length of', length)
    lengths.extend(lengths)
    freq_step = abs(freqs_L[:, 1] - freqs_L[:, 0])
    freq_span = abs(freqs_L[:, -1] - freqs_L[:, 0])
    for name, arr in [("Step", freq_step)]:
        print(f"--- {name} ---")
        print("  count:", arr.size)
        print("    mean:    ", np.mean(arr) / 1e6, "Mhz")
        print("    std:     ", np.std(arr) / 1e6, "Mhz")
        print("    min/max: ", np.min(arr) / 1e6, "Mhz", np.max(arr) / 1e6, "Mhz")
        print("    median:  ", np.median(arr) / 1e6, "Mhz")
        print("    25/75%:  ", np.percentile(arr, [25, 75]) / 1e6, "Mhz")
        print()
    for name, arr in [("Span", freq_span)]:
        print(f"--- {name} ---")
        print("  count:", arr.size)
        print("    mean:    ", np.mean(arr) / 1e6, "Mhz")
        print("    std:     ", np.std(arr) / 1e6, "Mhz")
        print("    min/max: ", np.min(arr) / 1e6, "Mhz", np.max(arr) / 1e6, "Mhz")
        print("    median:  ", np.median(arr) / 1e6, "Mhz")
        print("    25/75%:  ", np.percentile(arr, [25, 75]) / 1e6, "Mhz")
        print()

Analyzing channel length of 64
--- Step ---
  count: 8346
    mean:     31.25 Mhz
    std:      0.0 Mhz
    min/max:  31.25 Mhz 31.25 Mhz
    median:   31.25 Mhz
    25/75%:   [31.25 31.25] Mhz

--- Span ---
  count: 8346
    mean:     1968.75 Mhz
    std:      0.0 Mhz
    min/max:  1968.75 Mhz 1968.75 Mhz
    median:   1968.75 Mhz
    25/75%:   [1968.75 1968.75] Mhz

Analyzing channel length of 120
--- Step ---
  count: 9814
    mean:     15.625 Mhz
    std:      0.0 Mhz
    min/max:  15.625 Mhz 15.625 Mhz
    median:   15.625 Mhz
    25/75%:   [15.625 15.625] Mhz

--- Span ---
  count: 9814
    mean:     1859.375 Mhz
    std:      0.0 Mhz
    min/max:  1859.375 Mhz 1859.375 Mhz
    median:   1859.375 Mhz
    25/75%:   [1859.375 1859.375] Mhz

Analyzing channel length of 128
--- Step ---
  count: 5600
    mean:     15.625 Mhz
    std:      0.0 Mhz
    min/max:  15.625 Mhz 15.625 Mhz
    median:   15.625 Mhz
    25/75%:   [15.625 15.625] Mhz

--- Span ---
  count: 5600
    mean:     19

# Experimentation

In [ ]:
trans_df = pd.read_parquet(f'{cosmicai_path}/full_spectrum.gzip')

In [ ]:
df['frequency_array'] = df['frequency_array'] \
    .apply(lambda freqs: [f/1e9 for f in freqs])

In [ ]:
df['frequency_array']

In [ ]:
trans_freqs = trans_df['Frequency (GHz)'].values
trans_vals  = trans_df['Transmission (%)'].values

In [ ]:
def match_and_correct(freq_array):
    idxs = np.searchsorted(trans_freqs, freq_array)
    idxs[idxs == len(trans_freqs)] = len(trans_freqs) - 1
    left  = np.maximum(idxs - 1, 0)
    right = idxs
    dl = np.abs(freq_array - trans_freqs[left])
    dr = np.abs(trans_freqs[right] - freq_array)
    nearest = np.where(dl <= dr, left, right)
    mt = trans_vals[nearest]
    return mt

In [ ]:
results = df.apply(
    lambda row: match_and_correct(
        np.array(row['frequency_array'], dtype=float)
    ),
    axis=1
)

In [ ]:
df['transmission_array'] = results

In [ ]:
plt.figure(figsize=(100, 20))
plt.plot(trans_freqs, trans_vals, marker='o')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Transmission (%)')
plt.title('Transmission vs Frequency')
plt.grid(True)
plt.show()

In [ ]:
interference = []
for index in range(len(df)):
    freqs = np.array(df.iloc[index, 11])
    trans = np.array(df.iloc[index, 14])

    troguhs, props = find_peaks(-trans, prominence=1)
    widths_samples, _, left_ips, right_ips = peak_widths(-trans, troguhs, rel_height=0.75)

    left_freqs  = np.interp(left_ips,  np.arange(len(freqs)), freqs)
    right_freqs = np.interp(right_ips, np.arange(len(freqs)), freqs)
    widths_freq = right_freqs - left_freqs

    trough_freqs  = freqs[troguhs]
    trough_depths = props['prominences']

    trough_ranges = []
    for i in range(len(trough_freqs)):
        trough_ranges.append((trough_freqs[i] - widths_freq[i] / 2, trough_freqs[i] + widths_freq[i] / 2))

    trough_ranges = np.array(trough_ranges)

    closest_idxs = []

    for troguhs_range in trough_ranges:
        start, end = troguhs_range[0], troguhs_range[1]
        closest_start_idx = int(np.abs(freqs - start).argmin())
        closest_end_idx = int(np.abs(freqs - end).argmin())
        closest_idxs.append((closest_start_idx, closest_end_idx))

    interference.append(closest_idxs)

In [ ]:
index = 300
freqs = np.array(df.iloc[index, 11])
trans = np.array(df.iloc[index, 14])
# coeffs = np.polyfit(freqs, trans, 2)
# poly = np.poly1d(coeffs)
# fitted = poly(freqs)

plt.figure(figsize=(8,5))
plt.plot(range(len(freqs)), trans, 'o', label='Data')
# plt.plot(freqs, fitted, '-', label='Degree-3 fit')
plt.xlabel('Frequency (GHz)')
plt.ylabel('Transmission (%)')
plt.title('Quadratic Fit to Transmission')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
peaks_df = pd.read_parquet(f'{cosmicai_path}/all_peaks_widths_heights.gzip')

In [ ]:
peaks  = peaks_df['Peaks (GHz)'].values
widths = peaks_df['Widths'].values
peak_ranges = np.column_stack((peaks - widths / 2, peaks + widths / 2))
peak_ranges_mid_points = np.array([float(start + end) / 2 for start, end in peak_ranges])

In [ ]:
freq_ranges = np.array([
    [float(min(x)), float(max(x))]
    for x in df['frequency_array'].tolist()
])
freq_ranges_mid_points = np.array([float(start + end) / 2 for start, end in freq_ranges])

In [ ]:
for idx, freq_points in enumerate(freq_ranges_mid_points):
    closest_idx = int(np.abs(peak_ranges_mid_points - freq_points).argmin())
    freqs = freq_ranges[idx]
    peaks = peak_ranges[closest_idx]
    start = max(freqs[0], peaks[0])
    end   = min(freqs[1], peaks[1])

In [ ]:
df['atmospheric_interference'] = interference

In [ ]:
df.to_csv(
    f'{cosmicai_path}/bandpass_augmented.csv',
    index=False
)

# Debugging

In [ ]:
cosmicai_path = 'Data'

df_scored = pd.read_parquet(f'{cosmicai_path}/bandpass_qa0_no_partitions_labelled_filt_scored.parquet')

In [ ]:
df_scored.info()

In [ ]:
df_scored['win_span'] = ((df_scored['win_end'] - df_scored['win_start']).abs() * (df_scored['frequency_array'].str[1] - df_scored['frequency_array'].str[0])).abs()

In [ ]:
df_scored.info()

In [ ]:
df_scored_filtered_special = df_scored[df_scored['win_span'] == 31250000.0]

In [ ]:
df_sorted_special = (
    df_scored_filtered_special
    .assign(freq_len = df_scored_filtered_special['frequency_array'].str.len())
    .sort_values(
        by=[ 'freq_len', 'score' ],
        ascending=[ True,    False ]
    )
    .drop(columns='freq_len')
    .reset_index(drop=True)
)

In [ ]:
per_fig = 10
out_dir = 'Images/Special'
os.makedirs(out_dir, exist_ok=True)

In [ ]:
for length, grp in df_sorted_special.groupby(
        df_sorted_special['frequency_array'].str.len()
    ):
    n = len(grp)
    n_figs = math.ceil(n / per_fig)
    chunk = grp.iloc[fig_i*per_fig:(fig_i+1)*per_fig]
    fig_k = len(chunk)
    for fig_i in range(n_figs):
        chunk = grp.iloc[fig_i*per_fig:min((fig_i+1)*per_fig, len(grp))]
        fig_k = len(chunk)
        fig, axes = plt.subplots(fig_k, 1, figsize=(10, 3*fig_k))
        if fig_k == 1:
            axes = [axes]
        for ax, row in zip(axes, chunk.itertuples()):
            spec = row.amplitude_corr_tsys
            a, b = row.win_start, row.win_end
            atm_interfs = ast.literal_eval(row.atmospheric_interference)

            if len(atm_interfs) != 0:
                for item in atm_interfs:
                    c, d = item[0], item[1]
                    ax.axvspan(c, d, color='C9', alpha=0.2)

            x = np.arange(len(spec))
            ax.plot(x, spec, color='C0', label="Actual")

            ax.axvspan(a, b, color='C1', alpha=0.3)

            ax.set_title(
                f"eb_uid={row.eb_uid}, antenna={row.antenna}, polarization={row.polarization}, spw_name_ms={row.spw_name_ms}, score={row.score:.2f}, Range=[{a},{b}]"
            )
            ax.set_xlabel("Channel")
            ax.set_ylabel("Amplitude")
            ax.legend()
        plt.tight_layout(rect=[0,0,1,0.92])
        plt.suptitle(f"Items {fig_i*per_fig+1}–{fig_i*per_fig+fig_k}.", y=0.98)
        outpath = os.path.join(out_dir, f"length_{length}_fig{fig_i+1}.png")
        plt.savefig(outpath, dpi=500, bbox_inches="tight")
        plt.close(fig)
        if fig_i == 9:
            break

# Result Exploration

In [ ]:
df_scored = pd.read_parquet(f'science_scored.parquet')

In [ ]:
df_scored.info()

In [ ]:
PRIORITY = {
    "bandpass_amplitude_frequency",
    "bandpassflag_amplitude_frequency"        # ← add more as needed
}

def qa2_flag_code(flags):
    """
    Return 0 / 1 / 2  (no flags / priority flag / other flag).

    * priority = any list element whose lower-case *contains* a PRIORITY token
    * robust to lists, tuples, numpy arrays, or stringified lists
    """
    import ast, numpy as np, pandas as pd

    # ---- normalise to Python list ---------------------------------
    if flags is None or (isinstance(flags, float) and pd.isna(flags)):
        lst = []
    elif isinstance(flags, (list, tuple)):
        lst = list(flags)
    elif isinstance(flags, np.ndarray):
        lst = flags.tolist()
    else:
        s = str(flags).strip()
        if s in ("", "[]", "None"):
            lst = []
        else:                                # try to parse "['flag', ...]"
            try:
                parsed = ast.literal_eval(s)
                lst = list(parsed) if isinstance(parsed, (list, tuple, np.ndarray)) else [s]
            except Exception:
                lst = [s]

    # clean + lower-case for comparison
    clean = [str(f).strip().lower() for f in lst]

    # ---- classify -------------------------------------------------
    if not clean:
        return 0

    if any(any(token in f for token in PRIORITY) for f in clean):
        return 1
    return 2

In [ ]:
df_scored["qa2flag"] = df_scored["QA2 Flag(s)"].apply(qa2_flag_code)

In [ ]:
bins = np.round(np.arange(0.0, 1.0001, 0.05), 2)
labels = [f"{bins[i]:.2f}–{bins[i+1]:.2f}" for i in range(len(bins)-1)]

In [ ]:
df_scored = df_scored.copy()
df_scored["score_bin"] = pd.cut(df_scored["score"], bins=bins, labels=labels, include_lowest=True, right=True)

counts = (df_scored.loc[df_scored["qa2flag"] == 0]
            .groupby("score_bin")
            .size()
            .reindex(labels, fill_value=0))

totals = df_scored.groupby("score_bin").size().reindex(labels, fill_value=0)
rates = (counts / totals).fillna(0)

In [ ]:
counts

In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
counts.plot(kind="bar", ax=ax, rot=90)
ax.set_xlabel("Score bin")
ax.set_ylabel("Count with qa2flag = 0")
ax.set_title("QA2=0 counts by 0.05 score bins")
ax.bar_label(ax.containers[0], labels=[str(int(v)) for v in counts.values], padding=3)
plt.tight_layout()
plt.show()

In [ ]:
df_scored_false_negative = df_scored[(df_scored['qa2flag'] == 1) & (df_scored['score'] <= 0.3)]

In [ ]:
df_scored_false_negative.info()

In [ ]:
df_scored_false_negative = df_scored_false_negative.copy()

In [ ]:
df_scored_false_negative.drop(columns=['score', 'win_start', 'win_end', 'score_bin'], inplace=True)

In [ ]:
df_scored_false_negative.to_parquet('Data/false_negative.parquet')

In [ ]:
def plot_df(df_slice, per_fig=10, k=None, out_dir="Images/false_negative_anaysis"):
    df_slice = df_slice.copy()
    df_slice['orig_idx'] = df_slice.index
    df_slice['atmospheric_interference'] = df_slice['atmospheric_interference'].apply(ast.literal_eval)
    df_slice = df_slice.rename(columns={"QA2 Flag(s)": "QA2_Flags"})


    n_figs = math.ceil(len(df_slice) / per_fig)
    for fig_i in range(n_figs):
        chunk = df_slice.iloc[fig_i * per_fig : (fig_i + 1) * per_fig]
        fig_k = len(chunk)
        fig, axes = plt.subplots(fig_k, 1, figsize=(10, 3 * fig_k))
        if fig_k == 1:
            axes = [axes]

        for ax, row in zip(axes, chunk.itertuples(index=False)):
            spec = np.array(row.amplitude_corr_tsys)
            buffer = spec.shape[0] // 20
            a, b = row.win_start, row.win_end
            
            if len(row.atmospheric_interference) != 0:
                for (c, d) in row.atmospheric_interference:
                    ax.axvspan(c, d, color='C9', alpha=0.2)

            x = np.arange(len(spec))
            ax.plot(x, spec, color='C0', label="Actual")

            if buffer > 0:
                ax.axvspan(0, buffer-1, color='gray', alpha=0.2)
                ax.axvspan(len(spec)-buffer, len(spec)-1, color='gray', alpha=0.2)

            ax.axvspan(a, b, color='C1', alpha=0.3)

            ax.set_title(
                f"eb_uid={row.eb_uid}, antenna={row.antenna}, spw_name_ms={row.spw_name_ms}, polarization={row.polarization}, score={row.score:.2f}, range=[{a},{b}], qa0 flags={row.QA2_Flags}"
            )
            ax.set_xlabel("Channel")
            ax.set_ylabel("Amplitude")
            ax.legend()

        plt.tight_layout(rect=[0, 0, 1, 0.92])
        plt.suptitle(
            f"Items {fig_i * per_fig + 1}–{fig_i * per_fig + fig_k} of Top {k}.",
            y=0.98
        )
        outpath = os.path.join(out_dir, f"top_{k}_fig{fig_i+1}.png")
        plt.savefig(outpath, dpi=300, bbox_inches="tight")
        plt.close(fig)

In [ ]:
plot_df(
    df_scored_false_negative,
    per_fig=8,
    k=len(df_scored_false_negative)
)

# Dataset Creation

In [ ]:
ROOT = Path("Data/variable_length_kernel")
OUT  = "Data/spotcheck.csv"
SEED = 42
ORIGINAL_PATH = Path("Data/bandpass_qa0_no_partitions_labelled_filt.parquet")

In [ ]:
TARGET_LENGTHS = {64, 120, 128, 240, 256, 480, 960, 1024, 1920, 2048}

In [ ]:
def load_original() -> pd.DataFrame:
    if 'df_original' in globals() and isinstance(df_original, pd.DataFrame):
        return df_original
    if ORIGINAL_PATH.suffix == ".parquet":
        return pd.read_parquet(ORIGINAL_PATH)
    elif ORIGINAL_PATH.suffix == ".csv":
        return pd.read_csv(ORIGINAL_PATH)
    raise ValueError("Set ORIGINAL_PATH to a .parquet or .csv, or define df_original in memory.")


In [ ]:
def infer_length_from_dir(p: Path) -> int:
    m = re.search(r"length_(\d+)", str(p))
    if not m:
        raise ValueError(f"Cannot infer length from: {p}")
    return int(m.group(1))

In [ ]:
def filter_original_by_length(df: pd.DataFrame, L: int) -> pd.DataFrame:
    if "length" in df.columns:
        return df[df["length"] == L]
    for col in ("amplitude_corr_tsys", "spec_arrays"):
        if col in df.columns:
            return df[df[col].map(len) == L]
    raise KeyError("Could not infer length—provide a 'length' column or an array-like column.")

In [ ]:
def pick_top10(score_df: pd.DataFrame) -> pd.DataFrame:
    if "score" in score_df.columns:
        top10 = score_df.sort_values("score", ascending=False, kind="mergesort").head(10).copy()
    else:
        top10 = score_df.head(10).copy()
    top10['frequency_array'] = top10['frequency_array'].apply(lambda s: np.array(ast.literal_eval(s), dtype=float))
    top10['frequency_array'] = top10['frequency_array'].apply(lambda freqs: [f*1e9 for f in freqs])
    top10["sample_group"] = "top10"
    return top10

In [ ]:
def pick_random10_from_original(orig_len_df: pd.DataFrame, exclude_uids: set) -> pd.DataFrame:
    pool = orig_len_df[~orig_len_df["uid"].isin(exclude_uids)]
    k = min(10, len(pool))
    out = pool.sample(n=k, random_state=SEED, replace=False).copy()
    out["sample_group"] = "random_original"
    return out

In [ ]:
df_full = load_original()
if "uid" not in df_full.columns:
    df_full = df_full.reset_index().rename(columns={"index": "uid"})

samples = []

In [ ]:
for length_dir in ROOT.glob("length_*"):
    L = infer_length_from_dir(length_dir)
    csvs = sorted(length_dir.glob("*.csv"))
    if not csvs:
        continue
    score_path = csvs[0]  # or choose by name if you have a convention
    score_df = pd.read_csv(score_path)

    if "uid" not in score_df.columns:
        raise KeyError(f"'uid' missing in {score_path}")

    top10 = pick_top10(score_df)

    orig_len_df = filter_original_by_length(df_full, L)
    rand10 = pick_random10_from_original(orig_len_df, exclude_uids=set(top10["uid"]))

    samples.append(pd.concat([top10, rand10], ignore_index=True))

In [ ]:
final = pd.concat(samples, ignore_index=True)
final.drop(columns=['uid', 'transmission_array', 'atmospheric_interference', 'score', 'kernel_size', 'win_start', 'win_end', 'sample_group'], inplace=True)

In [ ]:
final.info()

In [ ]:
final.to_csv(OUT, index=False, sep='|')